In [1]:
import tensorflow as tf

In [3]:
with tf.Session() as sess:
    print("Hello")

AttributeError: module 'tensorflow' has no attribute 'Session'

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11264566011526649576
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4849834393
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10500998229170270625
physical_device_desc: "device: 0, name: GeForce GTX 1660 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
import tensorflow as tf
if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [81]:
!pip install sklearn

Processing c:\users\ashuk\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e\sklearn-0.0-py2.py3-none-any.whl
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)


In [100]:
# module import block of code
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image
from keras import layers
from keras.layers import Dropout,Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, MaxPool2D
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [42]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission_df = pd.read_csv("sample_submission.csv")

In [43]:
shape_x, shape_y = 1365, 2048

In [44]:
image_path = "images/"

In [45]:
# load train image
train_image_list = []
for image_name in train_df["image_id"]:
    image = Image.open("".join([image_path, image_name,".jpg"]))
    image = image.resize((80,80))
    #image = img.imread("".join([image_path, image_name,".jpg"]))
    train_image_list.append(np.asarray(image))

In [46]:
len(train_image_list)

1821

In [47]:
# load train image
test_image_list = []
for image_name in test_df["image_id"]:
    image = Image.open("".join([image_path, image_name,".jpg"]))
    image = image.resize((80,80))
    #image = img.imread("".join([image_path, image_name,".jpg"]))
    test_image_list.append(np.asarray(image))

In [48]:
len(test_image_list)

1821

In [52]:
X_Train = np.ndarray(shape=(len(train_image_list), 80, 80, 3),dtype = np.float32)
i=0
for image in train_image_list:
    #X_Train[i]=img_to_array(image)
    X_Train[i]=train_image_list[i]
    i=i+1
X_Train=X_Train/255

print('Train Shape: {}'.format(X_Train.shape))

Train Shape: (1821, 80, 80, 3)


In [54]:
X_Test = np.ndarray(shape=(len(test_image_list), 80, 80, 3),dtype = np.float32)
i=0
for image in test_image_list:
    #X_Test[i]=img_to_array(image)
    X_Test[i]=test_image_list[i]
    i=i+1
    
X_Test=X_Test/255
print('Test Shape: {}'.format(X_Test.shape))

Test Shape: (1821, 80, 80, 3)


In [56]:
print("Train shape: {}, Test shape: {}".format(X_Train.shape, X_Test.shape))

Train shape: (1821, 80, 80, 3), Test shape: (1821, 80, 80, 3)


In [58]:
del train_image_list
del test_image_list

In [67]:
y_train = train_df[train_df.keys().drop("image_id")].values
print(y_train.shape)

(1821, 4)


In [88]:
keras.__version__
img_size = 80

In [93]:
def construct_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', input_shape=(img_size, img_size, 3),
                activation='relu'))
    model.add(BatchNormalization()) # Normalize the activations of the previous layer at each batch
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
   
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Flatten()) # Flatten the input
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    # Configure the learning process
    # The loss function is the objective that the model will try to minimize
    # For any classification problem, use accuracy metric
    optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    model.summary()
    return model

In [94]:
X_train, X_val, Y_train, Y_val = train_test_split(X_Train, y_train, test_size=0.2, random_state=42)

In [99]:
model = construct_model()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 80, 80, 32)        896       
_________________________________________________________________
batch_normalization_15 (Batc (None, 80, 80, 32)        128       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 80, 80, 32)        9248      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 80, 80, 32)        9248      
_________________________________________________________________
batch_normalization_16 (Batc (None, 80, 80, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 40, 40, 32)       

In [101]:
datagen = ImageDataGenerator(rotation_range=360, # Degree range for random rotations
                        width_shift_range=0.2, # Range for random horizontal shifts
                        height_shift_range=0.2, # Range for random vertical shifts
                        zoom_range=0.2, # Range for random zoom
                        horizontal_flip=True, # Randomly flip inputs horizontally
                        vertical_flip=True) # Randomly flip inputs vertically

datagen.fit(X_train)

In [104]:
hist = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),
               steps_per_epoch=X_train.shape[0] // 32,
               epochs=120,
               verbose=1,
               validation_data=(X_val, Y_val))

Epoch 1/120
45/45 [==============================] - 13s 287ms/step - loss: 1.4993 - accuracy: 0.3265 - val_loss: 1.3973 - val_accuracy: 0.3288
Epoch 2/120
45/45 [==============================] - 4s 86ms/step - loss: 1.3115 - accuracy: 0.3294 - val_loss: 1.3931 - val_accuracy: 0.3479
Epoch 3/120
45/45 [==============================] - 4s 87ms/step - loss: 1.2672 - accuracy: 0.3441 - val_loss: 1.3704 - val_accuracy: 0.3342
Epoch 4/120
45/45 [==============================] - 4s 87ms/step - loss: 1.2566 - accuracy: 0.3504 - val_loss: 1.3935 - val_accuracy: 0.2740
Epoch 5/120
45/45 [==============================] - 4s 87ms/step - loss: 1.2560 - accuracy: 0.3441 - val_loss: 1.3982 - val_accuracy: 0.2740
Epoch 6/120
45/45 [==============================] - 4s 87ms/step - loss: 1.2387 - accuracy: 0.3820 - val_loss: 1.2557 - val_accuracy: 0.3479
Epoch 7/120
45/45 [==============================] - 4s 87ms/step - loss: 1.2390 - accuracy: 0.3926 - val_loss: 1.3257 - val_accuracy: 0.3479
Epoc

Epoch 115/120
45/45 [==============================] - 4s 99ms/step - loss: 0.2501 - accuracy: 0.9194 - val_loss: 0.3120 - val_accuracy: 0.8904
Epoch 116/120
45/45 [==============================] - 4s 98ms/step - loss: 0.2348 - accuracy: 0.9305 - val_loss: 0.3617 - val_accuracy: 0.8877
Epoch 117/120
45/45 [==============================] - 4s 98ms/step - loss: 0.2459 - accuracy: 0.9206 - val_loss: 0.5694 - val_accuracy: 0.7973
Epoch 118/120
45/45 [==============================] - 4s 98ms/step - loss: 0.2204 - accuracy: 0.9291 - val_loss: 0.5770 - val_accuracy: 0.7671
Epoch 119/120
45/45 [==============================] - 4s 99ms/step - loss: 0.2763 - accuracy: 0.9129 - val_loss: 0.2873 - val_accuracy: 0.9041
Epoch 120/120
45/45 [==============================] - 4s 99ms/step - loss: 0.2717 - accuracy: 0.9105 - val_loss: 0.5704 - val_accuracy: 0.8137


In [108]:
predict = model.predict(X_Test)
all_predict = np.ndarray(shape = (test_df.shape[0],4),dtype = np.float32)
for i in range(0,test_df.shape[0]):
    for j in range(0,4):
        if predict[i][j]==max(predict[i]):
            all_predict[i][j] = 1
        else:
            all_predict[i][j] = 0 
healthy = [y_test[0] for y_test in all_predict]
multiple_diseases = [y_test[1] for y_test in all_predict]
rust = [y_test[2] for y_test in all_predict]
scab = [y_test[3] for y_test in all_predict]
df = {'image_id':test_df.image_id,'healthy':healthy,'multiple_diseases':multiple_diseases,'rust':rust,'scab':scab}
data = pd.DataFrame(df)
data.to_csv("submission_5l.csv", index=False)